In [1]:
# Import libs
import praw
from tqdm import tqdm
import json

In [2]:
# Initializating reddit account for parsing
# Private info changed to ####
reddit = praw.Reddit(
    client_id="####",
    client_secret="####",
    user_agent="####",
    ratelimit_seconds = 4
)
 
# Function to preprocess texts
def string_fix(s):
    return s.replace("\n"," ").replace("\r"," ").replace('"','\\"')

In [3]:
# Loading files
data = open("data.json", 'a', encoding = "utf-8")
id_dict = open("id_list.json", 'r', encoding = "utf-8")
ids = json.load(id_dict)
id_dict.close()

# Selecting subreddits to parse and their listings (based on time)
subreddits = ['nottheonion', 'upliftingnews', 'offbeat', 'news', 'inthenews', 'neutralnews']
time_filters = ['all','year','month']

# Parsing process
for subr in subreddits:

    # Define the subreddit to parse: nottheonion, upliftingnews, offbeat, news, inthenews, neutralnews
    subreddit = reddit.subreddit(subr)

    for timef in time_filters:

        # time_filter – Can be one of: "all", "day", "hour", "month", "week", or "year" (default: "all")
        posts = subreddit.top(time_filter = timef, limit = None)

        for i, post in tqdm(enumerate(posts)):

            # Making sure we have not parsed the post yet (by id)
            if post.id not in ids["ids"]:

                # Each parsed post should have at least 20 comments. It ensures that the post is useful for the emotional recognition
                if post.num_comments >= 20:
                    
                    # Adding the post to the dictionaty of visited posts
                    ids["ids"].append(post.id)

                    # Sorting comments by upvotes. post.comments by default only returns top-level comments (in terms of comments tree), this is what we want
                    post.comment_sort = 'top'
                    i = 0
                    k = 1
                    z = 1

                    # Excluding moderator's comments, as they always have 1 upvote
                    if post.comments[0].score == 1:
                        i += 1
                    
                    try:
                        # Excluding deleted (by user) or removed (by moderation) comments
                        while post.comments[i].body == '[deleted]' or post.comments[i+k].body == '[deleted]' or post.comments[i+k+z].body == '[deleted]' or post.comments[i].body == '[removed]' or post.comments[i+k].body == '[removed]' or post.comments[i+k+z].body == '[removed]':
                            if post.comments[i].body == '[deleted]' or post.comments[i].body == '[removed]':
                                i += 1
                            if post.comments[i+k].body == '[deleted]' or post.comments[i+k].body == '[removed]':
                                k += 1
                            if post.comments[i+k+z].body == '[deleted]' or post.comments[i+k+z].body == '[removed]':
                                z += 1
                    
                    # If there are less then 3 top-level comments available, we skip that post
                    except IndexError:
                        continue
                    
                    # Filling the line with the needed data about the post
                    line = '{' + '\"subreddit\": \"{0}\", '\
                    '\"id\": \"{1}\", '\
                    '\"title\": \"{2}\", '\
                    '\"score\": {3}, '\
                    '\"upvote_ratio\": {4}, '\
                    '\"url\": \"{5}\", '\
                    '\"comment_1\": \"{6}\", '\
                    '\"comment_2\": \"{7}\", '\
                    '\"comment_3\": \"{8}\", '\
                    '\"score_c1\": {9}, '\
                    '\"score_c2\": {10}, '\
                    '\"score_c3\": {11}'\
                    .format(
                    post.subreddit,
                    post.id,
                    string_fix(post.title),
                    post.score,
                    post.upvote_ratio,
                    post.url,
                    string_fix(post.comments[i].body),
                    string_fix(post.comments[i+k].body),
                    string_fix(post.comments[i+k+z].body),
                    post.comments[i].score,
                    post.comments[i+k].score,
                    post.comments[i+k+z].score
                    ) + '}\n'

                    # Appending that line to a file
                    data.write(line)

977it [01:02, 15.64it/s]
999it [00:18, 54.24it/s]
144it [00:12, 11.94it/s]
993it [00:21, 47.06it/s]
999it [00:22, 43.83it/s]
349it [00:06, 50.05it/s]
993it [00:10, 96.46it/s] 
1000it [00:11, 90.67it/s]
214it [00:03, 67.12it/s]
993it [00:28, 34.78it/s]
998it [00:27, 36.52it/s]
925it [00:52, 17.75it/s]
998it [30:15,  1.82s/it]
1000it [11:57,  1.39it/s]
771it [05:07,  2.51it/s]
1000it [23:41,  1.42s/it]
1000it [09:28,  1.76it/s]
231it [00:03, 65.22it/s]


In [4]:
# Saving results
id_dict = open("id_list.json", 'w', encoding = "utf-8")
json.dump(ids, id_dict)
id_dict.close()
data.close()